<a href="https://colab.research.google.com/github/whitebird118/bluetooth_keyboard/blob/master/EMV_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Load CSV data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial provides examples of how to use CSV data with TensorFlow.

There are two main parts to this:

1. **Loading the data off disk**
2. **Pre-processing it into a form suitable for training.**

This tutorial focuses on the loading, and gives some quick examples of preprocessing. For a tutorial that focuses on the preprocessing aspect see the [preprocessing layers guide](https://www.tensorflow.org/guide/keras/preprocessing_layers#quick_recipes) and [tutorial](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers). 


## Setup

In [320]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

## In memory data

For any small CSV dataset the simplest way to train a TensorFlow model on it is to load it into memory as a pandas Dataframe or a NumPy array. 


A relatively simple example is the [abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone). 

* The dataset is small. 
* All the input features are all limited-range floating point values. 

Here is how to download the data into a [Pandas `DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [411]:
abalone_train = pd.read_csv(
    "https://raw.githubusercontent.com/whitebird118/bluetooth_keyboard/master/entire.csv",
    names=["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74","75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90","91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106","107","108","109","110","111","112","113","114","115","116","117","118","119","120","121","122","123","124","125","126","127","128","129","130","131","132","133","134","135","136","137","138","139","140","141","142","143","144","145","146","147","148","149","150","151","152","153","154","155","156","157","158","159","160","161","162","163","164","165","166","167","168","169","170","171","172","173","174","175","176","177","178","179","180","181","182","183","184","185","186","187","188","189","190","191","192","193","194","195","196","197","198","199","200","201","Label"])

abalone_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,Label
0,5.93,0.60,10.00,4.93,5.24,9.88,3.25,1.47,0.73,3.61,4.28,2.19,3.59,4.02,6.10,0.17,1.70,9.15,2.86,3.65,4.63,0.41,0.36,3.78,0.41,3.41,3.41,2.03,1.23,4.75,0.20,1.33,3.53,1.15,1.70,4.63,4.47,0.99,4.51,3.37,...,0.54,0.13,8.29,1.15,3.77,5.49,3.49,0.48,7.19,1.05,2.43,8.29,2.76,0.11,0.97,1.27,1.35,7.19,1.29,3.79,0.24,1.54,0.72,1.46,1.78,0.99,4.14,0.90,0.48,2.92,1.17,0.11,1.70,0.05,2.45,0.85,0.90,0.72,4.63,1
1,18.97,14.29,4.77,21.07,10.63,7.70,4.71,2.94,5.49,1.29,0.01,6.95,5.20,2.94,3.90,0.44,2.92,2.19,1.66,1.70,4.02,2.64,3.04,1.70,1.05,1.23,1.35,3.74,2.94,2.07,0.78,0.23,7.19,1.76,0.38,5.61,0.17,0.36,7.44,4.47,...,1.51,1.33,6.46,1.29,2.45,4.27,3.49,2.19,7.93,2.88,3.43,3.41,0.93,1.09,5.97,1.39,0.48,4.39,1.91,0.13,2.56,1.15,0.36,4.51,1.88,2.94,3.53,3.13,0.60,4.75,3.00,0.60,4.27,1.05,3.31,0.50,1.78,1.60,3.90,1
2,9.23,12.20,15.14,3.47,7.68,15.25,3.34,0.72,3.90,4.35,2.57,1.11,1.66,0.50,5.36,1.02,0.84,7.68,3.37,0.97,11.22,0.68,0.60,3.53,3.00,2.82,0.73,0.32,0.13,5.36,2.88,0.23,2.19,3.00,2.82,0.36,4.71,0.74,0.62,7.76,...,2.64,0.50,1.95,0.05,2.07,4.88,3.71,2.31,1.34,0.44,2.94,1.21,4.59,2.57,0.24,1.63,2.43,7.32,1.63,1.84,2.19,0.93,4.14,7.32,0.29,3.53,7.07,0.90,0.60,8.29,2.88,0.36,3.29,3.25,0.60,3.65,0.41,3.77,6.46,1
3,9.69,8.17,11.10,13.62,8.55,11.96,9.69,2.55,11.47,0.29,1.84,6.83,5.57,1.84,4.63,0.44,1.35,5.24,0.05,1.60,6.71,1.05,2.07,6.22,1.27,0.01,4.88,1.15,0.74,8.66,2.64,1.82,1.46,2.52,1.33,5.12,0.54,0.23,1.70,2.52,...,1.17,0.13,1.95,0.90,1.46,4.39,3.13,0.25,5.61,3.74,4.28,3.29,2.15,5.01,0.73,1.15,0.25,3.17,5.93,4.28,4.02,0.93,1.70,5.12,2.76,3.18,5.24,0.68,1.09,2.68,2.37,1.21,5.85,2.03,0.74,5.73,5.08,1.84,2.57,1
4,30.10,11.85,0.48,8.10,0.11,9.39,1.05,5.75,1.96,1.51,0.74,5.24,0.29,0.01,4.75,2.27,0.36,5.00,1.39,4.75,4.02,2.03,1.60,2.80,1.27,0.25,6.83,1.78,1.82,8.42,0.78,3.53,6.83,2.03,4.04,3.29,1.66,3.18,4.14,1.15,...,2.64,0.01,1.82,0.66,0.97,3.53,3.83,1.82,4.27,2.88,2.08,3.53,0.29,3.18,5.24,8.13,1.11,2.68,2.03,1.82,5.61,4.59,1.11,2.07,0.17,0.86,6.22,2.52,1.23,5.00,1.02,1.82,4.75,2.76,0.38,0.24,1.54,2.82,3.41,1


The nominal task for this dataset is to predict the age from the other measurements, so separate the features and labels for training:


In [412]:
abalone_features = abalone_train.copy()
abalone_labels = abalone_features.pop('Label')


For this dataset you will treat all features identically. Pack the features into a single NumPy array.:

In [413]:
abalone_features = np.array(abalone_features)

abalone_features


array([[ 5.93,  0.6 , 10.  , ...,  0.9 ,  0.72,  4.63],
       [18.97, 14.29,  4.77, ...,  1.78,  1.6 ,  3.9 ],
       [ 9.23, 12.2 , 15.14, ...,  0.41,  3.77,  6.46],
       ...,
       [11.42,  2.89, 50.07, ...,  0.66,  1.42,  1.57],
       [23.87, 16.64, 27.12, ...,  2.49,  1.79,  1.85],
       [ 3.24,  1.42,  5.88, ...,  0.78,  2.03,  0.38]])

Next make a regression model predict the age. Since there is only a single input tensor, a `keras.Sequential` model is sufficient here.

In [414]:
abalone_model = tf.keras.Sequential([
  layers.Dense(67, activation='relu'),
  layers.Dense(3, activation='softmax')
])

abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam(),
                      metrics = ['accuracy'])

To train that model, pass the features and labels to `Model.fit`:

In [415]:
abalone_model.fit(abalone_features, abalone_labels, epochs=100)

Epoch 1/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3952 - accuracy: 0.1594
Epoch 2/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3958 - accuracy: 0.2580
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3745 - accuracy: 0.2754
Epoch 4/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3565 - accuracy: 0.2464
Epoch 5/100
11/11 [==============================] - 0s 3ms/step - loss: 3.3387 - accuracy: 0.2899
Epoch 6/100
11/11 [==============================] - 0s 4ms/step - loss: 3.3212 - accuracy: 0.2667
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3067 - accuracy: 0.2435
Epoch 8/100
11/11 [==============================] - 0s 3ms/step - loss: 3.2979 - accuracy: 0.2493
Epoch 9/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2854 - accuracy: 0.2406
Epoch 10/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2788 - accuracy: 0.2551
Epoch 11/

You have just seen the most basic way to train a model using CSV data. Next, you will learn how to apply preprocessing to normalize numeric columns.

## Basic preprocessing

It's good practice to normalize the inputs to your model. The `experimental.preprocessing` layers provide a convenient way to build this normalization into your model. 

The layer will precompute the mean and variance of each column, and use these to normalize the data.

First you create the layer:

In [416]:
normalize = preprocessing.Normalization()

Then you use the `Normalization.adapt()` method to adapt the normalization layer to your data.

Note: Only use your training data to `.adapt()` preprocessing layers. Do not use your validation or test data.

In [417]:
normalize.adapt(abalone_features)

Then use the normalization layer in your model:

In [418]:
norm_abalone_model = tf.keras.Sequential([
  normalize,
  layers.Dense(67, activation='relu'),
  layers.Dense(3, activation='softmax')
])

norm_abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam(),
                           metrics=['accuracy'])

norm_abalone_model.fit(abalone_features, abalone_labels, epochs=100)


Epoch 1/100
11/11 [==============================] - 0s 2ms/step - loss: 3.3113 - accuracy: 0.1797
Epoch 2/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2817 - accuracy: 0.2087
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2663 - accuracy: 0.2435
Epoch 4/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2573 - accuracy: 0.2464
Epoch 5/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2521 - accuracy: 0.2493
Epoch 6/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2490 - accuracy: 0.2348
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2467 - accuracy: 0.2348
Epoch 8/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2449 - accuracy: 0.2261
Epoch 9/100
11/11 [==============================] - 0s 3ms/step - loss: 3.2440 - accuracy: 0.2203
Epoch 10/100
11/11 [==============================] - 0s 2ms/step - loss: 3.2431 - accuracy: 0.2377
Epoch 11/